# RES Forecasting - Data Presentation

The Greek RES dataset combines information from different sources. It contains geolocation, weather and RES energy production data of the top contributors of wind and solar energy that contribute the 20% of the RES power in the Greek energy grid. The weather data were collected from a weather forecast API and represent the last generated forecast for each installation.

The data were collected from the following sources:

* RES energy production from [ENTSO-E](https://transparency.entsoe.eu/load-domain/r2/totalLoadR2/show)
* Weather data captured using the [Stormglass API](https://docs.stormglass.io/#/weather?id=point-request)
* Wind turbine and PV installations geolocation data, for the locations that aquired operation licence as found in [Regulatory Authority for Energy (RAE)](https://geo.rae.gr/)

More information regarding energy licencing http://www.opengov.gr/minenv/?p=1031

---

**NOTE:** This notebook presents the process of the combination of the different types of data 

In [ ]:
import json
import pandas as pd
from datetime import timedelta, datetime
import os
import numpy as np
import re
import seaborn as sns
from tqdm import tqdm


## 1. Location data

Import the location data that contains the latitude and longtitude information of the most significant wind and solar power generation stations in Greece.
* id: location ID
* lon: longtitude
* lat: latidute
* type: energy type {wind, solar}


In [ ]:
location_df = pd.read_csv(os.path.join('resources', 'location_data.csv'), index_col=0)
location_df

## 2. Weather data

The location information should be used to collect weather data for the respective locations, that will be used to compile an aggregated weather dataset with the average values of the selected weather variables for the period of interest.

The most relevant weather variables captured by the [Stormglass API](https://docs.stormglass.io/#/weather?id=point-request) are the following:


| Time 	| Location 	|     Temperature    	|     Solar    	|        Wind       	|      Misc     	|
|:----:	|:--------:	|:------------------:	|:------------:	|:-----------------:	|:-------------:	|
| time 	| id1      	| airTemperature     	| cloudCover   	| windDirection     	| gust          	|
|      	| lat      	| airTemperature80m  	| currentSpeed 	| windDirection20m  	| precipitation 	|
|      	| lon      	| airTemperature100m 	| visibility   	| windDirection30m  	| humidity      	|
|      	|          	|                    	|              	| windDirection40m  	| pressure      	|
|      	|          	|                    	|              	| windDirection50m  	|               	|
|      	|          	|                    	|              	| windDirection80m  	|               	|
|      	|          	|                    	|              	| windDirection100m 	|               	|
|      	|          	|                    	|              	| windSpeed         	|               	|
|      	|          	|                    	|              	| windSpeed100m     	|               	|
|      	|          	|                    	|              	| windSpeed20m      	|               	|
|      	|          	|                    	|              	| windSpeed30m      	|               	|
|      	|          	|                    	|              	| windSpeed40m      	|               	|
|      	|          	|                    	|              	| windSpeed50m      	|               	|
|      	|          	|                    	|              	| windSpeed80m      	|               	|


More information on the weather variable can be found in the [Stormglass API Documentation](https://docs.stormglass.io/#/weather?id=point-request)

Lets load the combined historical weather data for all designated locations:

In [ ]:
weather_df = pd.read_csv('resources/raw_data/weather_data/historic_weather_data.20170101-20210102.csv', 
                         index_col=0,
                         parse_dates=['time'])
weather_df

Data preprocessing methods

In [ ]:
def set_na_values(df):
   '''
   Sets unfilled values or certain string as NA.
   '''
   return df.replace(to_replace={'n/e': np.nan, '-': np.nan})

def drop_na_values(df):
   '''
   Drops columns with less than 80% non-NA values
   '''
   return df.dropna(axis=1, thresh=int(len(df) * 0.8))
    
def drop_column(df, columns):
   return df.drop(columns=columns)

def set_time(df):
   df['time'] = pd.to_datetime(df['MTU'].str.split('-', expand=True)[0].str.strip())
   df.loc[:, 'time'] = df['time'].dt.tz_localize('CET', nonexistent=timedelta(hours=1), ambiguous=False)
   df.drop(columns=['MTU'], inplace=True)
   df.reset_index(inplace=True)
   df.drop(columns='index', inplace=True)
   return df

def to_numeric(df):
   '''
   Convert data to numeric.
   '''
   for col in set(df.columns) - set(['time']):
      df.loc[:, col] = pd.to_numeric(df[col])   
   return df

def na_fill(df, column):
   '''
   Fills missing values of the input column with the values of similar past days.
   '''
   na_dates = sorted(set(df.loc[df[column].isna()].index.date))
   for day in na_dates:
      orig_day_range = pd.date_range(start=floor_date(day), end=ceil_date(day), freq='H', tz='CET')
      similar_days = [day.replace(year=year) for year in list(set(df.index.year) - set([day.year]))]
      _df_list = []
      for sim_day in similar_days:
         day_range = pd.date_range(start=floor_date(sim_day), end=ceil_date(sim_day), freq='H', tz='CET')
         _df_list.append(df.loc[df.index.isin(day_range), column])
      _df = pd.concat(_df_list).to_frame()
      _df['hour'] = _df.index.time
      df.loc[df.index.isin(orig_day_range), column] = _df.groupby('hour').mean().values[:len(df[df.index.isin(orig_day_range)])]
   return df

def floor_date(date):
   '''
   Round date to the start of the day.
   '''
   date = pd.to_datetime(date)
   return date - timedelta(hours=date.time().hour, 
                         minutes=date.time().minute, 
                         seconds=date.time().second, 
                         microseconds=date.time().microsecond)

def ceil_date(date):
   '''
   Round date to the end of the day.
   '''
   date = floor_date(date)   
   return date + timedelta(hours=23)

### 2.1 Wind weather data preprocessing

Wind weather data preprocessing:

In [ ]:
wind_energy_df = weather_df.query('energy_type==\"wind\"').copy()
wind_energy_df.shape

In [ ]:
wind_energy_df = wind_energy_df.pipe(drop_na_values)
wind_energy_df.shape

In [ ]:
na_value_columns = [column for column in wind_energy_df.columns if wind_energy_df[column].isna().sum() > 0]

for column in na_value_columns:
   print(column)
   progressbar = tqdm(wind_energy_df.index.unique(), desc='Processing')

   for i in progressbar:
      progressbar.set_description('Processing location ID: %s' % i)
      df = wind_energy_df.loc[i].set_index('time')
      df = df.pipe(na_fill, column)
      
wind_energy_df.shape

In [ ]:
wind_energy_df = wind_energy_df.groupby('time').mean()
wind_energy_df.head()

### 2.1 Solar weather data preprocessing

Solar weather data preprocessing:

In [ ]:
solar_energy_df = weather_df.query('energy_type==\"solar\"').copy()
solar_energy_df.shape

In [ ]:
solar_energy_df = solar_energy_df.pipe(drop_na_values)
solar_energy_df.shape

In [ ]:
na_value_columns = [column for column in solar_energy_df.columns if solar_energy_df[column].isna().sum() > 0]

for column in na_value_columns:
   print(column)
   progressbar = tqdm(solar_energy_df.index.unique(), desc='Processing')

   for i in progressbar:
      progressbar.set_description('Processing location ID: %s' % i)
      df = solar_energy_df.loc[i].set_index('time')
      df = df.pipe(na_fill, column)
      
solar_energy_df.shape

In [ ]:
solar_energy_df = solar_energy_df.groupby('time').mean()
solar_energy_df.head()

## 3. Energy data
### 3.1 Energy data loading

Actual energy output data, at hourly intervals aggregated per energy type, were collected by the European Network of Transmission System Operators for Electricity ([ENTSO-E](https://transparency.entsoe.eu/load-domain/r2/totalLoadR2/show)), that represents 42 electricity transmission system operators (TSOs) from 35 countries across Europe.

In [ ]:
energy_data_dir = os.path.join('resources/raw_data/', 'ENTSOE_data')

# Combine all energy data in the directory 
energy_df = pd.concat([pd.read_csv(os.path.join(energy_data_dir, csv), 
                                   parse_dates=['MTU']) for csv in os.listdir(energy_data_dir)])
print(energy_df.shape)
energy_df.head()

In [ ]:
# Preprocessing
energy_df = (energy_df.pipe(set_na_values) # Define NA values
                      # Parse time column
                      .pipe(set_time) 
                      # Drop irrelevant columns
                      .pipe(drop_column, ['Area'] + list(set(energy_df.filter(regex='.+Actual.+').columns) 
                                                        - set(energy_df.filter(like='Wind').columns) 
                                                        - set(energy_df.filter(like='Solar').columns)))
                      # Drop NA values
                      .pipe(drop_na_values)
                      # 
                      .pipe(to_numeric))
energy_df.tail()

In [ ]:
energy_df = energy_df.set_index('time').sort_index().rename(columns={energy_df.columns[0]: 'solar power', 
                                                                     energy_df.columns[1]: 'wind power'})
energy_df.tail()

## 3.2 Align data according to time

Weather data are captrured at `UTC` timezone whereas the energy data are at the local `CET` timezone, thus the weather data are translated to `CET` time. Moreover, time missalignment in the datasets, caused due to DST time changes should be handled.

In [ ]:
energy_df.index = energy_df.index.tz_convert('CET')
energy_df.tail()

In [ ]:
solar_energy_df.index = solar_energy_df.index.tz_convert('CET')
solar_energy_df.tail()

In [ ]:
wind_energy_df.index = wind_energy_df.index.tz_convert('CET')
wind_energy_df.tail()

Define start and end period

In [ ]:
period_start = max(energy_df.index[0], 
                   solar_energy_df.index[0], 
                   wind_energy_df.index[0])
period_end = min(energy_df.index[-1], 
                    solar_energy_df.index[-1], 
                    wind_energy_df.index[-1])
period_start, period_end

Trim data according to start and end of the period

In [ ]:
energy_df = energy_df[(period_start < energy_df.index) & 
                      (energy_df.index < period_end)]
energy_df.head()

In [ ]:
solar_energy_df = solar_energy_df[(period_start < solar_energy_df.index) & 
                                  (solar_energy_df.index < period_end)]
solar_energy_df.head()

In [ ]:
wind_energy_df = wind_energy_df[(period_start < wind_energy_df.index) & 
                                  (wind_energy_df.index < period_end)]
wind_energy_df.head()

Resample data at hourly intervals

In [ ]:
energy_df = energy_df.resample('1h').mean()
solar_energy_df = solar_energy_df.resample('1h').mean()
wind_energy_df = wind_energy_df.resample('1h').mean()

## 3.3 Wind energy dataset preperation

In [ ]:
wind_power_df = pd.concat([wind_energy_df, energy_df['wind power'].to_frame()], axis=1)
wind_power_df = wind_power_df.interpolate(method='pad')
wind_power_df.head()

## 3.3 Solar energy dataset preperation

In [ ]:
solar_power_df = pd.concat([solar_energy_df, energy_df['solar power'].to_frame()], axis=1)
solar_power_df = solar_power_df.interpolate(method='pad')
solar_power_df.head()

In [ ]:
processed_data_dir = os.path.join('resources', 'processed_data')

if not os.path.exists(processed_data_dir):
   os.mkdir(processed_data_dir)

wind_power_df.to_csv(os.path.join(processed_data_dir, 'wind_energy_w_forecast.csv')) 
solar_power_df.to_csv(os.path.join(processed_data_dir, 'solar_energy_w_forecast.csv'))